# Poisson problem with Uniform refinement.


In [ ]:
#Poisson problem with homogeneous Dirichlet boundary conditions using residual estimator and Dorfler marking

from ngsolve import *
import netgen.gui
from netgen.geom2d import SplineGeometry
from math import *
import numpy as np
%matplotlib inline



#   point numbers 0, 1, ... 11
#   sub-domain numbers (1), (2), (3)
#  
#
#             7-------------6
#             |             |
#             |     (2)     |
#             |             |
#      3------4-------------5------2
#      |                           |
#      |             11            |
#      |           /   \           |
#      |         10 (3) 9          |
#      |           \   /     (1)   |
#      |             8             |
#      |                           |
#      0---------------------------1
#

def MakeGeometry():
    geometry = SplineGeometry()
    
    # point coordinates ...
    pnts = [ (0,0), (1,0), (1,0.6), (0,0.6), \
             (0.2,0.6), (0.8,0.6), (0.8,0.8), (0.2,0.8), \
             (0.5,0.15), (0.65,0.3), (0.5,0.45), (0.35,0.3) ]
    pnums = [geometry.AppendPoint(*p) for p in pnts]
    
    # start-point, end-point, boundary-condition, domain on left side, domain on right side:
    lines = [ (0,1,1,1,0), (1,2,2,1,0), (2,5,2,1,0), (5,4,2,1,2), (4,3,2,1,0), (3,0,2,1,0), \
              (5,6,2,2,0), (6,7,2,2,0), (7,4,2,2,0), \
              (8,9,2,3,1), (9,10,2,3,1), (10,11,2,3,1), (11,8,2,3,1) ]
        
    for p1,p2,bc,left,right in lines:
        geometry.Append( ["line", pnums[p1], pnums[p2]], bc=bc, leftdomain=left, rightdomain=right)
    return geometry



mesh = Mesh(MakeGeometry().GenerateMesh (maxh=0.2))
n = specialcf.normal(2)
h = specialcf.mesh_size


fes = H1(mesh, order=3, dirichlet=[1], autoupdate=True)
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx

c = MultiGridPreconditioner(a, inverse = "sparsecholesky")
gfu = GridFunction(fes, autoupdate=True)
Draw (gfu)


f = LinearForm(fes)
fcoeff= CoefficientFunction(1)
f+=fcoeff*v*dx




def SolveBVP():
    a.Assemble()
    f.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data = inv * f.vec
    Redraw (blocking=True)


l = []

def hess(w): 
    return w.Operator("hesse")
def Lap(w):
   return hess(w)[0,0]+hess(w)[1,1]

def DorflerMarking(elerr):
        DorflerParam=0.25
        NpElError = np.array(elerr)
        SortedElError = np.sort(NpElError)[::-1]
        inds = np.argsort(NpElError)[::-1]
        Total = np.sum(NpElError)
        S=0
        counter=0
        while S<DorflerParam*Total:
            S=S+SortedElError[counter]
            counter=counter+1   
        for el in mesh.Elements():
            mesh.SetRefinementFlag(el, True) 
            for i in range(0,counter):
                el = mesh[ElementId(VOL,inds[i])]
                type(el)
                mesh.SetRefinementFlag(el,True)
 


def CalcError():
    err=(h*h)*((fcoeff+Lap(gfu))**2)*dx
    elerr = Integrate (err, mesh, element_wise=True)
    elerr += Integrate(h*0.5*((grad(gfu)-grad(gfu).Other())*n)**2*dx(element_boundary=True),mesh,element_wise=True)
    l.append ( (fes.ndof, sqrt(sum(elerr)) ))
    return elerr

    
    


with TaskManager():
    while fes.ndof < 10000:  
        SolveBVP()
        elerr=CalcError()
        DorflerMarking(elerr)
        mesh.Refine()
    
SolveBVP()



import matplotlib.pyplot as plt

plt.xlabel("log(ndof)")
plt.ylabel("log(H1 error-estimate)")
ndof,err = zip(*l)
plt.ion()
x=np.log10(np.array(ndof))
y=np.log10(np.array(err))
plt.plot(x,y, "-*")
m,b = np.polyfit(x, y, 1)
y1 = m*x+b

plt.plot(x,y1)

#plt.savefig('dorfler with uniform.png')
plt.show()


# Poisson problem with adaptive refinement


In [ ]:
#Poisson problem with homogeneous Dirichlet boundary conditions using residual estimator and Dorfler marking

from ngsolve import *
import netgen.gui
from netgen.geom2d import SplineGeometry
from math import *
import numpy as np
%matplotlib inline



#   point numbers 0, 1, ... 11
#   sub-domain numbers (1), (2), (3)
#  
#
#             7-------------6
#             |             |
#             |     (2)     |
#             |             |
#      3------4-------------5------2
#      |                           |
#      |             11            |
#      |           /   \           |
#      |         10 (3) 9          |
#      |           \   /     (1)   |
#      |             8             |
#      |                           |
#      0---------------------------1
#

def MakeGeometry():
    geometry = SplineGeometry()
    
    # point coordinates ...
    pnts = [ (0,0), (1,0), (1,0.6), (0,0.6), \
             (0.2,0.6), (0.8,0.6), (0.8,0.8), (0.2,0.8), \
             (0.5,0.15), (0.65,0.3), (0.5,0.45), (0.35,0.3) ]
    pnums = [geometry.AppendPoint(*p) for p in pnts]
    
    # start-point, end-point, boundary-condition, domain on left side, domain on right side:
    lines = [ (0,1,1,1,0), (1,2,2,1,0), (2,5,2,1,0), (5,4,2,1,2), (4,3,2,1,0), (3,0,2,1,0), \
              (5,6,2,2,0), (6,7,2,2,0), (7,4,2,2,0), \
              (8,9,2,3,1), (9,10,2,3,1), (10,11,2,3,1), (11,8,2,3,1) ]
        
    for p1,p2,bc,left,right in lines:
        geometry.Append( ["line", pnums[p1], pnums[p2]], bc=bc, leftdomain=left, rightdomain=right)
    return geometry



mesh = Mesh(MakeGeometry().GenerateMesh (maxh=0.2))
n = specialcf.normal(2)
h = specialcf.mesh_size


fes = H1(mesh, order=3, dirichlet=[1], autoupdate=True)
u = fes.TrialFunction()
v = fes.TestFunction()



a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx

c = MultiGridPreconditioner(a, inverse = "sparsecholesky")
gfu = GridFunction(fes, autoupdate=True)
Draw (gfu)



f = LinearForm(fes)
fcoeff= CoefficientFunction(1)
f+=fcoeff*v*dx




def SolveBVP():
    a.Assemble()
    f.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data = inv * f.vec
    Redraw (blocking=True)


l = []

def hess(w): 
    return w.Operator("hesse")
def Lap(w):
   return hess(w)[0,0]+hess(w)[1,1]

def DorflerMarking(elerr):
        DorflerParam=0.25
        NpElError = np.array(elerr)
        SortedElError = np.sort(NpElError)[::-1]
        inds = np.argsort(NpElError)[::-1]
        Total = np.sum(NpElError)
        S=0
        counter=0
        while S<DorflerParam*Total:
            S=S+SortedElError[counter]
            counter=counter+1   
        for el in mesh.Elements():
            mesh.SetRefinementFlag(el, False) 
            for i in range(0,counter):
                el = mesh[ElementId(VOL,inds[i])]
                type(el)
                mesh.SetRefinementFlag(el,True)
 


def CalcError():
    err=(h*h)*((fcoeff+Lap(gfu))**2)*dx
    elerr = Integrate (err, mesh, element_wise=True)
    elerr += Integrate(h*0.5*((grad(gfu)-grad(gfu).Other())*n)**2*dx(element_boundary=True),mesh,element_wise=True)
    l.append ( (fes.ndof, sqrt(sum(elerr)) ))
    return elerr

    
    


with TaskManager():
    while fes.ndof < 10000:  
        SolveBVP()
        elerr=CalcError()
        DorflerMarking(elerr)
        mesh.Refine()
    
SolveBVP()

#print(count, 'count')

import matplotlib.pyplot as plt

plt.xlabel("log(ndof)")
plt.ylabel("log(H1 error-estimate)")
ndof,err = zip(*l)
plt.ion()
x=np.log10(np.array(ndof))
y=np.log10(np.array(err))
plt.plot(x,y, "-*")
m,b = np.polyfit(x, y, 1)
y1 = m*x+b

plt.plot(x,y1)

#plt.savefig('dorfler with uniform.png')
plt.show()


# Checking Equidistribution for poisson problem  

In [ ]:
#checking equidistribution Poisson problem with homogeneous Dirichlet boundary conditions using residual estimator and Dorfler marking

from ngsolve import *
import netgen.gui
from netgen.geom2d import SplineGeometry
from math import *
import numpy as np
%matplotlib inline



#   point numbers 0, 1, ... 11
#   sub-domain numbers (1), (2), (3)
#  
#
#             7-------------6
#             |             |
#             |     (2)     |
#             |             |
#      3------4-------------5------2
#      |                           |
#      |             11            |
#      |           /   \           |
#      |         10 (3) 9          |
#      |           \   /     (1)   |
#      |             8             |
#      |                           |
#      0---------------------------1
#

def MakeGeometry():
    geometry = SplineGeometry()
    
    # point coordinates ...
    pnts = [ (0,0), (1,0), (1,0.6), (0,0.6), \
             (0.2,0.6), (0.8,0.6), (0.8,0.8), (0.2,0.8), \
             (0.5,0.15), (0.65,0.3), (0.5,0.45), (0.35,0.3) ]
    pnums = [geometry.AppendPoint(*p) for p in pnts]
    
    # start-point, end-point, boundary-condition, domain on left side, domain on right side:
    lines = [ (0,1,1,1,0), (1,2,2,1,0), (2,5,2,1,0), (5,4,2,1,2), (4,3,2,1,0), (3,0,2,1,0), \
              (5,6,2,2,0), (6,7,2,2,0), (7,4,2,2,0), \
              (8,9,2,3,1), (9,10,2,3,1), (10,11,2,3,1), (11,8,2,3,1) ]
        
    for p1,p2,bc,left,right in lines:
        geometry.Append( ["line", pnums[p1], pnums[p2]], bc=bc, leftdomain=left, rightdomain=right)
    return geometry



mesh = Mesh(MakeGeometry().GenerateMesh (maxh=0.2))
n = specialcf.normal(2)
h = specialcf.mesh_size


fes = H1(mesh, order=1, dirichlet=[1], autoupdate=True)
u = fes.TrialFunction()
v = fes.TestFunction()


a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx

c = MultiGridPreconditioner(a, inverse = "sparsecholesky")
gfu = GridFunction(fes, autoupdate=True)
Draw (gfu)


f = LinearForm(fes)
fcoeff= CoefficientFunction(1)
f+=fcoeff*v*dx




def SolveBVP():
    a.Assemble()
    f.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data = inv * f.vec
    Redraw (blocking=True)


l = []

def hess(w): 
    return w.Operator("hesse")
def Lap(w):
   return hess(w)[0,0]+hess(w)[1,1]

def DorflerMarking(elerr):
        DorflerParam=0.25
        NpElError = np.array(elerr)
        SortedElError = np.sort(NpElError)[::-1]
        inds = np.argsort(NpElError)[::-1]
        Total = np.sum(NpElError)
        S=0
        counter=0
        while S<DorflerParam*Total:
            S=S+SortedElError[counter]
            counter=counter+1   
        for el in mesh.Elements():
            mesh.SetRefinementFlag(el, False) 
            for i in range(0,counter):
                el = mesh[ElementId(VOL,inds[i])]
                type(el)
                mesh.SetRefinementFlag(el,True)
 

eqerr=[]

def CalcError(count,eqerr):

    err=(h*h)*((fcoeff+Lap(gfu))**2)*dx
    elerr = Integrate (err, mesh, element_wise=True)
    elerr += Integrate(h*0.5*((grad(gfu)-grad(gfu).Other())*n)**2*dx(element_boundary=True),mesh,element_wise=True)
    elerr = list(elerr)
    l.append ( (fes.ndof, sqrt(sum(elerr)) ))
    for i in elerr:
        eqerr.append(i)
    count=count+1
    return elerr, count, eqerr

    
    

count = 0
with TaskManager():
    while fes.ndof <1000:  
        SolveBVP()
        elerr,count,eqerr=CalcError(count,eqerr)
        DorflerMarking(elerr)
        mesh.Refine()
    
SolveBVP()



import matplotlib.pyplot as plt

mean=sum(eqerr)/len(eqerr)
y1=[mean]*len(eqerr)

eqerr = np.array(eqerr)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
from scipy import stats
from scipy.stats import norm

x=eqerr
hist, bins, _ = plt.hist(x, bins=10)
logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
plt.subplot(212)
plt.hist(x, bins=logbins)
plt.xscale('log')
plt.xlabel("local errors")
plt.savefig('Error Equidistribution.png')
plt.show()
dev=np.std(eqerr)
print('std deviation is',dev)



# Problem 1


In [ ]:
#-nabla(u)+u=1 with neumann boundary condition (grad u.n=0) using residual estimator and Dorfler marking
from ngsolve import *
import netgen.gui
from math import *
import numpy as np
from netgen.geom2d import unit_square
%matplotlib inline


# generate a triangular mesh of mesh-size 0.2
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))

n = specialcf.normal(2)
h = specialcf.mesh_size


fes = H1(mesh, order=3, autoupdate=True, dgjumps=True)
u = fes.TrialFunction()
v = fes.TestFunction()
errspace = L2(mesh, order = 0, autoupdate=True,dgjumps=True) 
etaerr = errspace.TrialFunction()
q = errspace.TestFunction()

a = BilinearForm(fes, symmetric=True)
aerr = BilinearForm(errspace, symmetric=True)
a += grad(u)*grad(v)*dx+u*v*dx
aerr += etaerr*q*dx

c = MultiGridPreconditioner(a, inverse = "sparsecholesky")
gfu = GridFunction(fes, autoupdate=True)
Draw (gfu)
gfuerr = GridFunction(errspace,autoupdate=True)


f = LinearForm(fes)
fcoeff= CoefficientFunction(1)
gcoeff= CoefficientFunction(1)
f+=fcoeff*v*dx+gcoeff*v*ds
Ferr = LinearForm(errspace)
Ferr+=h*((gcoeff-grad(gfu)*n)**2)* 0.5*(q+q.Other())*ds('1')
Ferr+=h*((gcoeff-grad(gfu)*n)**2)* 0.5*(q+q.Other())*ds('2')
Ferr+=h*((gcoeff-grad(gfu)*n)**2)* 0.5*(q+q.Other())*ds('3')
Ferr+=h*((gcoeff-grad(gfu)*n)**2)* 0.5*(q+q.Other())*ds('4')



def SolveBVP():
    a.Assemble()
    f.Assemble()
    aerr.Assemble()
    Ferr.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data = inv * f.vec
    gfuerr.vec.data = aerr.mat.Inverse(errspace.FreeDofs())*Ferr.vec
    Redraw (blocking=True)


l = []

def hess(w): 
    return w.Operator("hesse")
def Lap(w):
   return hess(w)[0,0]+hess(w)[1,1]

def DorflerMarking(elerr):
        DorflerParam=0.25
        NpElError = np.array(elerr)
        SortedElError = np.sort(NpElError)[::-1]
        inds = np.argsort(NpElError)[::-1]
        Total = np.sum(NpElError)
        S=0
        counter=0
        while S<DorflerParam*Total:
            S=S+SortedElError[counter]
            counter=counter+1   
        for el in mesh.Elements():
            mesh.SetRefinementFlag(el, False) 
            for i in range(0,counter):
                el = mesh[ElementId(VOL,inds[i])]
                type(el)
                mesh.SetRefinementFlag(el,True)
 


def CalcError():

    err=(h*h)*((fcoeff+Lap(gfu)-gfu)**2)*dx
    elerr = Integrate (err, mesh, element_wise=True)
    elerr += Integrate(gfuerr,mesh, VOL, element_wise=True)
    elerr += Integrate(h*0.5*((grad(gfu)-grad(gfu).Other())*n)**2*dx(element_boundary=True),mesh,element_wise=True)
    
    l.append ( (fes.ndof, sqrt(sum(elerr)) ))
    return elerr

    
    


with TaskManager():
    while fes.ndof < 10000:  
        SolveBVP()
        elerr=CalcError()
        DorflerMarking(elerr)
        mesh.Refine()
    
SolveBVP()


import matplotlib.pyplot as plt

plt.xlabel("log(ndof)")
plt.ylabel("log(H1 error-estimate)")
ndof,elerr = zip(*l)
plt.ion()
x=np.log10(np.array(ndof))
y=np.log10(np.array(elerr))
plt.plot(x,y, "-*")
m,b = np.polyfit(x, y, 1)
y1 = m*x+b

plt.plot(x,y1)

plt.savefig('Problem_1.png')
plt.show()


# Problem 2

In [ ]:
#-nabla(u)+u_x-u_y=1 in on L-shaped domain, with homogeneous Dirichlet boundary conditions (u=0 on boundary) using residual estimator and Dorfler marking
from ngsolve import *
import netgen.gui
from netgen.geom2d import SplineGeometry
from math import *
import numpy as np
%matplotlib inline


def MakeGeometry():
    geometry = SplineGeometry()
    
    # point coordinates ...
    pnts = [ (0,0), (1,0), (1,0.6), (0.5,0.6), (0.5, 0.8), (0, 0.8) ]
    pnums = [geometry.AppendPoint(*p) for p in pnts]
    
    # start-point, end-point, boundary-condition, domain on left side, domain on right side:
    lines = [ (0,1,1,1,0), (1,2,1,1,0), (2,3,1,1,0), (3,4,1,1,0), (4,5,1,1,0), (5,0,1,1,0)]
        
    for p1,p2,bc,left,right in lines:
        geometry.Append( ["line", pnums[p1], pnums[p2]], bc=bc, leftdomain=left, rightdomain=right)
    return geometry




mesh = Mesh(MakeGeometry().GenerateMesh (maxh=0.2))


n = specialcf.normal(2)
h = specialcf.mesh_size


fes = H1(mesh, order=3, dirichlet=[1], autoupdate=True)
u = fes.TrialFunction()
v = fes.TestFunction()


a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx+grad(u)[0]*v*dx-grad(u)[1]*v*dx

c = MultiGridPreconditioner(a, inverse = "sparsecholesky")
gfu = GridFunction(fes, autoupdate=True)
Draw (gfu)


f = LinearForm(fes)
fcoeff= CoefficientFunction(1)
f+=fcoeff*v*dx




def SolveBVP():
    a.Assemble()
    f.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data = inv * f.vec
    Redraw (blocking=True)


l = []

def hess(w): 
    return w.Operator("hesse")
def Lap(w):
   return hess(w)[0,0]+hess(w)[1,1]

def DorflerMarking(elerr):
        DorflerParam=0.25
        NpElError = np.array(elerr)
        SortedElError = np.sort(NpElError)[::-1]
        inds = np.argsort(NpElError)[::-1]
        Total = np.sum(NpElError)
        S=0
        counter=0
        while S<DorflerParam*Total:
            S=S+SortedElError[counter]
            counter=counter+1   
        for el in mesh.Elements():
            mesh.SetRefinementFlag(el, False) 
            for i in range(0,counter):
                el = mesh[ElementId(VOL,inds[i])]
                type(el)
                mesh.SetRefinementFlag(el,True)
 


def CalcError():

    err=(h*h)*((fcoeff+Lap(gfu)-grad(gfu)[0]+grad(gfu)[1])**2)*dx
    elerr = Integrate (err, mesh, element_wise=True)
    elerr += Integrate(h*0.5*((grad(gfu)-grad(gfu).Other())*n)**2*dx(element_boundary=True),mesh,element_wise=True)
    l.append ( (fes.ndof, sqrt(sum(elerr)) ))
    return elerr

    
    


with TaskManager():
    while fes.ndof < 10000:  
        SolveBVP()
        elerr=CalcError()
        DorflerMarking(elerr)
        mesh.Refine()
    
SolveBVP()



import matplotlib.pyplot as plt

plt.xlabel("log(ndof)")
plt.ylabel("log(H1 error-estimate)")
ndof,elerr = zip(*l)
plt.ion()
x=np.log10(np.array(ndof))
y=np.log10(np.array(elerr))
plt.plot(x,y, "-*")
m,b = np.polyfit(x, y, 1)
y1 = m*x+b

plt.plot(x,y1)

plt.savefig('problem_2.png')
plt.show()


# Problem 3

In [ ]:
#-(nabla(u))+b.grad(u)+u=1 in L-Shaped domain,b=(x,y) ,u=0 in T_D using residual estimator and  dorfler marking
from ngsolve import *
import netgen.gui
from netgen.geom2d import SplineGeometry
from math import *
import numpy as np
%matplotlib inline



def MakeGeometry():
    geometry = SplineGeometry()
    
    # point coordinates ...
    pnts = [ (0,0), (1,0), (1,0.6), (0.5,0.6), (0.5, 0.8), (0, 0.8) ]
    pnums = [geometry.AppendPoint(*p) for p in pnts]
    
    # start-point, end-point, boundary-condition, domain on left side, domain on right side:
    lines = [ (0,1,1,1,0), (1,2,1,1,0), (2,3,1,1,0), (3,4,1,1,0), (4,5,1,1,0), (5,0,1,1,0)]
        
    for p1,p2,bc,left,right in lines:
        geometry.Append( ["line", pnums[p1], pnums[p2]], bc=bc, leftdomain=left, rightdomain=right)
    return geometry



mesh = Mesh(MakeGeometry().GenerateMesh (maxh=0.2))
n = specialcf.normal(2)
h = specialcf.mesh_size


fes = H1(mesh, order=3, dirichlet=[1], autoupdate=True)
u = fes.TrialFunction()
v = fes.TestFunction()
b_t = CoefficientFunction((x,y))


a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx+np.dot(b_t, grad(u))*v*dx+u*v*dx

c = MultiGridPreconditioner(a, inverse = "sparsecholesky")
gfu = GridFunction(fes, autoupdate=True)
Draw (gfu)


f = LinearForm(fes)
fcoeff= CoefficientFunction(1)
f+=fcoeff*v*dx



def SolveBVP():
    a.Assemble()
    f.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data = inv * f.vec
    Redraw (blocking=True)


l = []

def hess(w): 
    return w.Operator("hesse")
def Lap(w):
   return hess(w)[0,0]+hess(w)[1,1]

def DorflerMarking(elerr):
        DorflerParam=0.25
        NpElError = np.array(elerr)
        SortedElError = np.sort(NpElError)[::-1]
        inds = np.argsort(NpElError)[::-1]
        Total = np.sum(NpElError)
        S=0
        counter=0
        while S<DorflerParam*Total:
            S=S+SortedElError[counter]
            counter=counter+1   
        for el in mesh.Elements():
            mesh.SetRefinementFlag(el, False) 
            for i in range(0,counter):
                el = mesh[ElementId(VOL,inds[i])]
                type(el)
                mesh.SetRefinementFlag(el,True)
 


def CalcError():
    dif=np.dot(b_t,grad(gfu))
    err=(h*h)*((fcoeff+Lap(gfu)-dif-gfu)**2)*dx
    elerr = Integrate (err, mesh, element_wise=True)
    elerr += Integrate(h*0.5*((grad(gfu)-grad(gfu).Other())*n)**2*dx(element_boundary=True),mesh,element_wise=True)
    l.append ( (fes.ndof, sqrt(sum(elerr)) ))
    return elerr

    
    


with TaskManager():
    while fes.ndof < 10000:  
        SolveBVP()
        elerr=CalcError()
        DorflerMarking(elerr)
        mesh.Refine()
    
SolveBVP()



import matplotlib.pyplot as plt

plt.xlabel("log(ndof)")
plt.ylabel("log(H1 error-estimate)")
ndof,elerr = zip(*l)
plt.ion()
x=np.log10(np.array(ndof))
y=np.log10(np.array(elerr))
plt.plot(x,y, "-*")
m,b = np.polyfit(x, y, 1)
y1 = m*x+b

plt.plot(x,y1)

#plt.savefig('Problem_3.png')
plt.show()

# Problem 4


In [ ]:
#-(nabla(u))+b.grad(u)=1 in omega,b=(-6y,6x) ,u=0 in T_D and grad(u).n=0 in T_N  using residual estimator and 
# dorfler marking
from ngsolve import *
import netgen.gui
from netgen.geom2d import SplineGeometry
from math import *
import numpy as np
%matplotlib inline

geo = SplineGeometry()
geo.AddCircle ( (0, 0), r=1, bc="circle")

mesh = Mesh(geo.GenerateMesh(maxh=0.2))
n = specialcf.normal(2)
h = specialcf.mesh_size


fes = H1(mesh, order=3, dirichlet="circle", autoupdate=True)
u = fes.TrialFunction()
v = fes.TestFunction()
b_t = CoefficientFunction((-6*y,6*x))

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx+np.dot(b_t, grad(u))*v*dx

c = MultiGridPreconditioner(a, inverse = "sparsecholesky")
gfu = GridFunction(fes, autoupdate=True)
Draw (gfu)


f = LinearForm(fes)
fcoeff= CoefficientFunction(1)
f+=fcoeff*v*dx




def SolveBVP():
    a.Assemble()
    f.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data = inv * f.vec
    Redraw (blocking=True)


l = []

def hess(w): 
    return w.Operator("hesse")
def Lap(w):
   return hess(w)[0,0]+hess(w)[1,1]

def DorflerMarking(elerr):
        DorflerParam=0.25
        NpElError = np.array(elerr)
        SortedElError = np.sort(NpElError)[::-1]
        inds = np.argsort(NpElError)[::-1]
        Total = np.sum(NpElError)
        S=0
        counter=0
        while S<DorflerParam*Total:
            S=S+SortedElError[counter]
            counter=counter+1   
        for el in mesh.Elements():
            mesh.SetRefinementFlag(el, False) 
            for i in range(0,counter):
                el = mesh[ElementId(VOL,inds[i])]
                type(el)
                mesh.SetRefinementFlag(el,True)
 


def CalcError():
    dif=np.dot(b_t,grad(gfu))
    err=(h*h)*((fcoeff+Lap(gfu)-dif)**2)*dx
    elerr = Integrate (err, mesh, element_wise=True)
    elerr += Integrate(h*0.5*((grad(gfu)-grad(gfu).Other())*n)**2*dx(element_boundary=True),mesh,element_wise=True)
    l.append ( (fes.ndof, sqrt(sum(elerr)) ))
    return elerr

    
    


with TaskManager():
    while fes.ndof < 10000:  
        SolveBVP()
        elerr=CalcError()
        DorflerMarking(elerr)
        mesh.Refine()
    
SolveBVP()



import matplotlib.pyplot as plt

plt.xlabel("log(ndof)")
plt.ylabel("log(H1 error-estimate)")
ndof,elerr = zip(*l)
plt.ion()
x=np.log10(np.array(ndof))
y=np.log10(np.array(elerr))
plt.plot(x,y, "-*")
m,b = np.polyfit(x, y, 1)
y1 = m*x+b

plt.plot(x,y1)

#plt.savefig('problem_4.png')
plt.show()


# Convergence with Z-Z Type estimator for model problem

In [ ]:
#Implementation of z-z type error estimator alongwith dorfler marking
from ngsolve import *
import netgen.gui
from netgen.geom2d import SplineGeometry
from math import *
import numpy as np
%matplotlib inline


#   point numbers 0, 1, ... 11
#   sub-domain numbers (1), (2), (3)
#  
#
#             7-------------6
#             |             |
#             |     (2)     |
#             |             |
#      3------4-------------5------2
#      |                           |
#      |             11            |
#      |           /   \           |
#      |         10 (3) 9          |
#      |           \   /     (1)   |
#      |             8             |
#      |                           |
#      0---------------------------1
#

def MakeGeometry():
    geometry = SplineGeometry()
    
    # point coordinates ...
    pnts = [ (0,0), (1,0), (1,0.6), (0,0.6), \
             (0.2,0.6), (0.8,0.6), (0.8,0.8), (0.2,0.8), \
             (0.5,0.15), (0.65,0.3), (0.5,0.45), (0.35,0.3) ]
    pnums = [geometry.AppendPoint(*p) for p in pnts]
    
    # start-point, end-point, boundary-condition, domain on left side, domain on right side:
    lines = [ (0,1,1,1,0), (1,2,2,1,0), (2,5,2,1,0), (5,4,2,1,2), (4,3,2,1,0), (3,0,2,1,0), \
              (5,6,2,2,0), (6,7,2,2,0), (7,4,2,2,0), \
              (8,9,2,3,1), (9,10,2,3,1), (10,11,2,3,1), (11,8,2,3,1) ]
        
    for p1,p2,bc,left,right in lines:
        geometry.Append( ["line", pnums[p1], pnums[p2]], bc=bc, leftdomain=left, rightdomain=right)
    return geometry



mesh = Mesh(MakeGeometry().GenerateMesh (maxh=0.2))


fes = H1(mesh, order=3, dirichlet=[1], autoupdate=True)
u = fes.TrialFunction()
v = fes.TestFunction()

lam = CoefficientFunction([1,1,1])
a = BilinearForm(fes, symmetric=False)
a += lam*grad(u)*grad(v)*dx


f = LinearForm(fes)
f += CoefficientFunction(1)*v*dx

c = MultiGridPreconditioner(a, inverse = "sparsecholesky")

gfu = GridFunction(fes, autoupdate=True)
Draw (gfu)

space_flux = HDiv(mesh, order=2, autoupdate=True)
gf_flux = GridFunction(space_flux, "flux", autoupdate=True)

def SolveBVP():
    a.Assemble()
    f.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data = inv * f.vec
    Redraw (blocking=True)



l = []
def DorflerMarking(elerr):
        DorflerParam=0.25
        NpElError = np.array(elerr)
        SortedElError = np.sort(NpElError)[::-1]
        inds = np.argsort(NpElError)[::-1]
        Total = np.sum(NpElError)
        S=0
        counter=0
        while S<DorflerParam*Total:
            S=S+SortedElError[counter]
            counter=counter+1   
        for el in mesh.Elements():
            mesh.SetRefinementFlag(el, False) 
            for i in range(0,counter):
                el = mesh[ElementId(VOL,inds[i])]
                type(el)
                mesh.SetRefinementFlag(el,True)

def CalcError():
    flux = lam * grad(gfu)
    gf_flux.Set (flux)
    # Gradient-recovery error estimator
    err = 1/lam*(flux-gf_flux)*(flux-gf_flux)
    elerr = Integrate (err, mesh, VOL, element_wise=True)
    l.append ( (fes.ndof, sqrt(sum(elerr)) ))
    return elerr

with TaskManager():
    while fes.ndof < 10000:  
        SolveBVP()
        elerr= CalcError()
        DorflerMarking(elerr)
        mesh.Refine()
    
SolveBVP()

import matplotlib.pyplot as plt

plt.xlabel("log(ndof)")
plt.ylabel("log(H1 error-estimate)")
ndof,elerr = zip(*l)
plt.ion()
x=np.log10(np.array(ndof))
y=np.log10(np.array(elerr))
plt.plot(x,y, "-*")
m,b = np.polyfit(x, y, 1)
y1 = m*x+b

plt.plot(x,y1)

plt.savefig('z-z_error_estimator.png')
plt.show()



# Checking Efficiency of our residual type error estimator

In [ ]:
#solving poisson equation on a square domain with homogeneous BC
from ngsolve import *
import netgen.gui
from math import *
import numpy as np
from netgen.geom2d import unit_square
%matplotlib inline


# generate a triangular mesh of mesh-size 0.2
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))

n = specialcf.normal(2)
h = specialcf.mesh_size


fes = H1(mesh, order=3, dirichlet=[1,2,3,4], autoupdate=True)
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx

c = MultiGridPreconditioner(a, inverse = "sparsecholesky")
gfu = GridFunction(fes, autoupdate=True)
Draw (gfu)
u_exact=x*(1-x)*y*(1-y)
du0=y*(1-y)*(1-2*x)
du1=x*(1-x)*(1-2*y)

f = LinearForm(fes)
fcoeff= 2 * (y*(1-y)+x*(1-x))
f+=fcoeff*v*dx




def SolveBVP():
    a.Assemble()
    f.Assemble()
    inv = CGSolver(a.mat, c.mat)
    gfu.vec.data = inv * f.vec
    Redraw (blocking=True)




def hess(w): 
    return w.Operator("hesse")
def Lap(w):
   return hess(w)[0,0]+hess(w)[1,1]

def DorflerMarking(elerr):
        DorflerParam=0.25
        NpElError = np.array(elerr)
        SortedElError = np.sort(NpElError)[::-1]
        inds = np.argsort(NpElError)[::-1]
        Total = np.sum(NpElError)
        S=0
        counter=0
        while S<DorflerParam*Total:
            S=S+SortedElError[counter]
            counter=counter+1   
        for el in mesh.Elements():
            mesh.SetRefinementFlag(el, False) 
            for i in range(0,counter):
                el = mesh[ElementId(VOL,inds[i])]
                type(el)
                mesh.SetRefinementFlag(el,True)
 
l = []
L = []
def d0(u): 
    return grad(u)[0]
def d1(u): 
    return grad(u)[1]


def CalcError():
    err=(h*h)*((fcoeff+Lap(gfu))**2)*dx
    elerr = Integrate (err, mesh, element_wise=True)
    elerr += Integrate(h*0.5*((grad(gfu)-grad(gfu).Other())*n)**2*dx(element_boundary=True), mesh, element_wise=True)
    print('estimated Error is', sqrt(sum(elerr)))
    l.append ( (fes.ndof, sqrt(sum(elerr)) ))
    return elerr
   
def H1Error():
    H1Err = Integrate((gfu-u_exact)*(gfu-u_exact), mesh, order=6)
    H1Err += Integrate((du0-d0(gfu))**2+(du1-d1(gfu))**2, mesh, order=6)
    print('H1 Error is', sqrt(H1Err))
    L.append ( (fes.ndof, sqrt(H1Err )))
    return H1Err

    


with TaskManager():
    while fes.ndof < 10000:  
        SolveBVP()
        elerr=CalcError()
        H1Err=H1Error()
        DorflerMarking(elerr)
        mesh.Refine()
    
SolveBVP()



import matplotlib.pyplot as plt


plt.xlabel("log(ndof)")
plt.ylabel("log(H1 error-estimate)")
ndof,elerr = zip(*l)
ndof,H1Err = zip(*L)
plt.ion()
x=np.log10(np.array(ndof))
y=np.log10(np.array(elerr))
y_1=np.log10(np.array(H1Err))
plt.plot(x,y, "-*")
plt.plot(x,y_1, "-*")
plt.legend(["Residual error estimate", "True Error"])
m,b = np.polyfit(x, y, 1)
m1,b1 = np.polyfit(x , y_1, 1)
y1 = m*x+b
y2 = m1*x+b1
plt.plot(x, y1)
plt.plot(x, y2)
xmax=max(x)
y1min=min(y)
y2min=min(y1)
plt.vlines(x=xmax, ymin=y2min, ymax=y1min)
#plt.savefig('Efficiency')
plt.show()
